In [ ]:
import sqlite3
import csv
from sqlalchemy import create_engine, select, MetaData, Table

# SQLite database and 'books' table
conn = sqlite3.connect('books.db')
cursor = conn.cursor()

# the books table
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    title TEXT,
    author TEXT,
    year INTEGER
)
''')
conn.commit()

# insert data from books2.csv
with open('books2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row if present
    for row in reader:
        cursor.execute('''
            INSERT INTO books (title, author, year)
            VALUES (?, ?, ?)
        ''', (row[0], row[1], int(row[2])))  # Assuming year is in the third column
conn.commit()

# close the SQLite connection
conn.close()

#  connect to the SQLite database and query data
engine = create_engine('sqlite:///books.db')
metadata = MetaData()
books = Table('books', metadata, autoload_with=engine)

# Query to select the title in alphabetical order
with engine.connect() as connection:
    stmt = select(books.c.title).order_by(books.c.title)
    results = connection.execute(stmt)

    print("Titles (using SQLAlchemy) in alphabetical order:")
    for row in results:
        print(row[0])

# Query to select all columns in order of publication year
with engine.connect() as connection:
    stmt = select(books).order_by(books.c.year)
    results = connection.execute(stmt)

    print("\nAll books in order of publication year:")
    for row in results:
        print(row)
